### Import Modules

In [20]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time


sys.path.append("../")
import fitter.data_loader as dl
import fitter.fit_manager as fm
import fitter.fitter as fit
import fitter.special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [21]:
from importlib import reload
reload(dl)
reload(sf)
reload(fit)
reload(fm)

<module 'fitter.fit_manager' from '../fitter/fit_manager.py'>

### Specify fit parameters

In [24]:
params = {
    'collection_name' : 'test', 
    
    'model_info' : {
        'fit_type' : 'xpt', # 'ma', 'ma-ratio', 'xpt', 'xpt-ratio', or 'poly'
        'order' : 'n2lo', # 'nlo', 'n2lo', 'n3lo'
        'latt_ct' : 'n4lo',
        'F2' : 'FpiFpi', #'FKFpi', 'FpiFpi', 'FKFK', 'F0F0' (HPQCD choice)

        'include_FV' : True, # max 10
        'exclude' : [], # put LECs here
        'use_bijnens_central_value' : True,
        
        # semi-nnlo corrections
        'include_alpha_s' : False,
        'include_log' : True, 
        'include_log2' : True,
        'include_sunset' : True,
    }, 

    'fast_susnet' : False,
    'use_prior' : False,
    'bias_correct' : True,
    #'include_su2_isospin_corrrection' : False,
    
    'abbrs' : [u'a06m310L', u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : True,
    
    'save_results' : False,
    'save_pickles' : True,
    'output_name' : 'fit_results',
}

In [25]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()
phys_point_data = data_loader.get_phys_point_data()
params['model_info']['name'] = data_loader.get_model_name_from_model_info(params['model_info'])

# Get prior
prior = None
if params['use_prior']:
    prior = data_loader.get_prior(**params['model_info'], collection_name=params['collection_name'])
    
# Make fit
fit_manager = fm.fit_manager(fit_data, phys_point_data, prior=prior, **params)

if params['make_plots']:
    data_loader.save_plots(
        fit_manager.make_plots(show_error_ellipses=params['show_error_ellipses']),
        output_filename=params['collection_name']+'/fits/'+fit_manager.model
    )
else:
    print(fit_manager)
    
if params['save_results']:
    data_loader.save_fit_info(
        fit_manager.fit_info,
        collection_name=params['collection_name'], 
        save_pickles=params['save_pickles']
    )

    
    
t1 = time.time()

print("\nTotal time (s): ", t1 - t0, "\n")

Using default prior.
Making fit...
Done!
Time (s):  20.736867666244507

Model: xpt_FpiFpi_n2lo_log_logSq_sunset_a4_a6_FV_bijnens

Fitted/[FLAG] values at physical point:
	F_K / F_pi = 1.2035(48) [1.1959(20)]	(delta_su2 = -0.00446(71))

Parameters:
            L_4    -0.0002 (29)      [   0.0000 (50) ]  
            L_5   -0.00043 (54)      [   0.0000 (50) ]  
            L_2    0.00032 (16)      [  0.00033 (16) ]  
            L_3    -0.0036 (15)      [  -0.0031 (15) ]  
            L_7   -0.00036 (17)      [ -0.00034 (17) ]  
            A_k      -0.9 (7.4)      [        0 (10) ]  
            A_p       2.3 (3.8)      [        0 (10) ]  
            A_a      -9.1 (2.4)      [       0 (100) ]  
           A_aa         78 (53)      [      0 (1000) ]  
          A_aaa    -1028 (6078)      [     0 (10000) ]  

Least Square Fit:
  chi2/dof [dof] = 0.65 [17]    Q = 0.85    logGBF = 63.115

Settings:
  svdcut/n = 1e-12/0    tol = (1e-08*,1e-10,1e-10)    (itns/time = 48/20.7)

Error Budget (r

In [5]:
print(fit_manager.fmt_error_budget())

Partial Errors:
                  FK/Fpi
------------------------
          A_a:   0.00010
         A_aa:   0.00020
        A_aaa:   0.00202
          A_k:   0.00004
          A_p:   0.00000
          L_1:   0.00003
          L_2:   0.00008
          L_3:   0.00032
          L_4:   0.00005
          L_5:   0.00003
          L_6:   0.00000
          L_7:   0.00001
          L_8:   0.00000
     phys:mpi:   0.00013
      phys:mk:   0.00017
phys:lam2_chi:   0.00140
         stat:   0.00407
------------------------
        total:   0.00478



In [18]:
fit_manager.plot_fit(param='mpi');

In [19]:
fit_manager.plot_fit(param='a');

In [34]:
fit_data = {
    'mpi' : 135.,
    'mk' : 495.5,
    'Fpi' : 92.2,
    'FK' : 110.5,
    'L' : np.inf, 
    'a/w0' : 0,
}
fit_data['lam2_chi'] = (4 *np.pi *fit_data['Fpi'])**2

fit_parameters = {
    'L_1' : 0.000372,
    'L_2' : 0.000493,
    'L_3' : -0.003070,
    'L_4' : 0.000089,
    'L_5' : 0.000377,
    'L_6' : 0.000011,
    'L_7' : -0.000340,
    'L_8' : 0.000294,
    'A_a' : 0.1,
    'A_k' : -3.000000,
    'A_p' : 4.000000
}


fit_manager.fk_fpi_fit_fcn(fit_data=fit_data, fit_parameters=fit_parameters, debug=True)

NameError: name 'bootstrapper' is not defined